# MP 5: Pick and Place

**Due date**: March 6, 2021 at 10:45am.

**Instructions**: Read and complete the problems below. In this assignment, you should be switched over to a local install.  As a backup, you may run code through this notebook via Binder, but be aware that you will be editing external .py files, and saving your progress is not as simple as clicking "Save notebook state" in this window.

To submit your assignment, perform the following:

1. Double-check that your programs run without error.
2. Send this file, all of your .py files and any other files you used in your programs on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).
3. If you are using any external libraries other than the ones that are indicated during the installation process, include a README file indicating which library and version you are using.  To be on the safe side, you should include a backup procedure until the graders verify that they are able to support use of that library.

**Important**: You will need to run `patch_a_pip_install.py` from Klampt source again...

## Problem 1: Basic motion planning

This problem will ask you to drive the Kinova Gen3 robot from configuration to configuration using the Klamp't sampling-based motion planning tools.  Consult the [Klamp't Motion Planning docs](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/Manual-Planning.html) for more information.

### Problem 1.A

Implement `feasible_plan` in `planning.py` using the [planToConfig](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.plan.robotplanning.html#klampt.plan.robotplanning.planToConfig) function of `klampt.plan.robotplanning` to generate a `MotionPlan` motion planner object configured to reach the target configuration.  Set this up so that only the links numbered 1,2,3,4,5,6,7 are included in the moving subset. (Note: if this returns None, your start or goal were infeasible.)

Then, repeatedly call `MotionPlan.planMore` with some small number of iterations (less than 100 or so) to invoke the planner for up to 10 seconds, or until `MotionPlan.getPath` returns a non-empty list.  You should play around with the keyword arguments as described by the documentation in order to get reasonable performance.  Try the keywords `type='sbl',perturbationRadius=0.5` to start.

When you run the `problem1.py` program, you should be able to edit the robot configuration by right-clicking or dragging on it.  Every time you press 'p' or select 'Plan' from the menu, the planner will begin to run.

### Problem 1.B

Now, you will examine how to configure optimizing planners. In `optimizing_plan`, implement the calls necessary to repeatedly call `MotionPlan.planMore` for the time limit of 10s.  Note that you should not break the loop when a first path is found.

The given goal configuration is a bit tricky for some of these planners to reach. In `problem1.py`, change the goal configuration from 'goal.config' to 'goal_easy.config' when PROBLEM='1b' is set.  Using the default start and the "easy" goal, run each of the following planner settings:

- `type='sbl',shortcut=True,perturbationRadius=0.5`: The SBL planner followed by shortcutting.
- `type='sbl',perturbationRadius=0.5,shortcut=True,restart=True,restartTermCond="{foundSolution:1,maxIters:100}"`: The SBL planner followed by shortcutting and random restarts.
- `type='rrt*'`: The RRT* planner.
- `type='lazyrrg*'`: An RRT* like strategy, but which performs lazy collision checking.

Note that you might find it convenient to store these settings in a Python dictionary and pass them to the `planToConfig` function as keyword arguments using the `**kwargs` operator.

Gather statistics for 10 runs about the success rate and final optimized path length at the end of that 10s planning duration.  You should place this raw data in a text or CSV file.  In the space below for written answers, report the mean and standard deviation for the above settings  If you desire, you may tweak the parameter settings as well, but if you do so, please report the settings that you used to generate this data.

## Problem 2: Pick motion planning

Problem 2 loads several grasps and their scores as defined by a grasp sampler similar to one that we used for MP3.  Your job is to find a transit path to one of them that has a high quality score (i.e., a low score).

### Problem 2.A

In `pick.py`, implement the `plan_pick_one()` function that takes a single grasp as input, and
1. Use IK to find a collision-free configuration.  The `ik.solve_global` function is helpful here, and it can take a collision checker as input.  (Don't forget to ignore collisions between the object and the finger pads).
2. Retract this configuration along the approach direction, opening the gripper to a wider configuration using `gripper.set_finger_config(qgrasp,gripper.partway_open_config(1))`. Check collisions along the open and approach.
3. Create a planner (using one of your preferred planner settings) to generate the transit path.  (You will want to fully open the gripper in the start configuration too.)
4. Lift the object slightly.
5. Returns a triple containing the transit path, approach, and lift.

Running `problem_2.py` will test your picking planner with a known reachable grasp.

### Problem 2.B

Now, you are given a set of candidate grasps.  Design the `plan_pick_iterate` algorithm to iterate through the grasps in order of increasing score, and try sampling a feasible IK configuration and planning a feasible motion for a fixed amount of time.  If you fail, move to the next grasp.

In the space below for written answers, describe the tradeoffs you would encounter when setting the planning time duration. What problems might result when this duration is low?  When this duration is high?

Do you observe any effects between the distance moved along approach direction and the planning time or path quality?

### Problem 2C

Setting `PROBLEM=2c` activates the `plan_pick_multistep` function, which calls the `PickPlanner` solver. The solver implements a working set technique, where each partial plan consists of:
- the grasp,
- the grasp configuration,
- the approach motion,
- the transit path,
- and the lift motion
The partial plan is a dictionary,  well as cached objects and information about how much time has been spent on each of them. 

Much of the skeleton is implemented for you; you just need to implement the individual components, and possibly a scoring function.

Is this approach significantly faster or more reliable than the result of Problem 2B?  Why or why not?

## Problem 3:Pick and Place Planning
 
### Problem 3A

In `place.py`, implement the `transfer_plan` function that plans a feasible transfer plan to a target configuration. A transfer motion planner must perform collision checking so that the object moves along with the robot's gripper according to the given grasp.  The easiest way to do this is to provide a callback to `extraConstraints` which will check collision between the object, the environment, and the robot.  Don't forget to avoid collision testing the object against the gripper! 

Test your code by running `problem3.py`.  Press 'p' to run your plan, and then press 'e' to execute it.

### Problem 3B

Implement the rest of the placing pipeline in the `PlacePlanner` class, which is a subclass of `MultiStepPlanner`.  Test this by setting `PROBLEM='3b'`

### Problem 3C

Integrate the `PickPlanner` and `PlacePlanner` so that you can use them to to clean the table in a while loop.  During your transfer motions, the object will move as though it were attached to the robot.  This can lead to really weird-looking behavior if you haven't implemented your IK solving correctly!

If all goes well, repeatedly pressing 'p' and 'e' will let you clean the table.

## Written responses

### Written response for Problem 1.B

Put your answer here.

### Written response for Problem 2.B

Put your answer here.

### Written response for Problem 2.C

Put your answer here.